In [41]:
import torch
from torch import nn, optim, utils
from torch.nn import functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models import vgg as vgg
import numpy as np


In [27]:
class build_model(nn.Module):
    def __init__(self):
        super().__init__()
        # network
        node_i = 784
        node_m = 512
        node_o = 10
        self.fm = nn.Linear(node_i, node_m)
        self.fo = nn.Linear(node_m, node_o)
        # loss function
        self.criterion = nn.MSELoss()
        # optimization function
        self.optimizer = optim.Adam(self.parameters())
        print ("model created")

    def forward(self, x):
        x = self.fm(x)
        # print ("middle layer calc done:", x)
        x = F.relu(x)
        # print ("middle layer activation done:", x)
        x = self.fo(x)
        # print ("out layer calc done:", x)
        x = F.relu(x)
        # print ("out layer activation done:", x)
        return x


In [29]:
def train(model, train_loader):
    # let model know it is for training
    model.train()

    total_correct = 0
    total_loss = 0
    total_data_len = 1
    accuracy = 0
    loss = 0

    # loop by minibatch
    for batch_imgs, batch_labels in train_loader:
        # convert 2d image data to 1d array
        batch_imgs = batch_imgs.reshape(-1, 28*28*1)
        # convert 10 elements array to one-hot array : only one answer data will be 1 and others becomes 0
        labels = torch.eye(10)[batch_labels]

        outputs = model(batch_imgs)
        model.optimizer.zero_grad()
        loss = model.criterion(outputs, labels)
        loss.backward()
        model.optimizer.step()

        # ミニバッチごとの正答率と損失を求める
        _, pred_labels = torch.max(outputs, axis=1)  # outputsから必要な情報(予測したラベル)のみを取り出す。
        batch_size = len(batch_labels)  # バッチサイズの確認
        for i in range(batch_size):  # データ一つずつループ,ミニバッチの中身出しきるまで
            total_data_len += 1  # 全データ数を集計
            if pred_labels[i] == batch_labels[i]:
                total_correct += 1 # 正解のデータ数を集計
        total_loss += loss.item()  # 全損失の合計

    # 今回のエポックの正答率と損失を求める
    accuracy = total_correct/total_data_len*100  # 予測精度の算出
    loss = total_loss/total_data_len  # 損失の平均の算出
    return accuracy, loss


In [56]:
def load_data():
    trainset = datasets.MNIST(root='./data', train=True, download=False, transform=transforms.ToTensor())
    testset = datasets.MNIST(root='./data', train=False, download=False, transform=transforms.ToTensor())
    train_loader = utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2) # for train, shuffle=True
    test_loader = utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2) # for train, shuffle=True
    print ("loading mnist done")
    return train_loader, test_loader


In [58]:
model = build_model()
train_loader, test_loader = load_data()

acc, loss = train(model, train_loader)
print(f'正答率: {acc}, 損失: {loss}')


model created
loading mnist done
正答率: 74.36209396510058, 損失: 0.00032055950027979707


In [43]:
    print ("torch version:", torch.__version__)
    print ("torchivision version:", torchvision.__version__)
    print ("numpy version:", np.__version__)


torch version: 2.3.0
torchivision version: 0.18.1a0
numpy version: 1.24.3


In [11]:
print (torch.__version__)
print (torchvision.__version__)
print (np.__version__)

2.3.0.post100
0.18.1a0
1.24.3


In [19]:
np.zeros((2,3))

array([[0., 0., 0.],
       [0., 0., 0.]])

In [6]:
torch.zeros((2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [29]:
np.array(([0,1,2],[3,4,5]))

array([[0, 1, 2],
       [3, 4, 5]])

In [33]:
torch.tensor(([0,1,2],[3,4,5]))

tensor([[0, 1, 2],
        [3, 4, 5]])

In [13]:
ones = torch.ones((2,3), dtype=torch.float64)
print (ones)
print (ones.shape, ones.dtype)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
torch.Size([2, 3]) torch.float64
